In [32]:
# for reading data
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import plotly as plt



example website
https://medium.com/luca-chuangs-bapm-notes/build-a-neural-network-in-python-multi-class-classification-e940f74bd899

In [33]:
#read in the data
#take input from provisional database (csv)
# Load the csv file from GitHub
url = 'https://raw.githubusercontent.com/mandymccabe/Final_Project/janet_branch/Data/Final_Project_Full.csv'
url2= 'https://raw.githubusercontent.com/mandymccabe/Final_Project/main/Resources/all_responses_coded.csv'
df = pd.read_csv(url, index_col=0)
df2 = pd.read_csv(url2, index_col=0)

In [34]:
clean_df2= df2.drop(['A1','A2','A3','A4','A5','A21','StartDate','EndDate'], axis=1)

In [35]:
PoliticalViews = df.filter(["political_views"], axis=1)
PoliticalViews.head()

,political_views
respondentid,
6176264298,Moderate
6176263960,Moderate
6176258621,Liberal
6176257082,Liberal
6176256111,Liberal


In [36]:
merged_dfs = pd.merge(clean_df2, PoliticalViews, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
merged_dfs.head()

,RespondentID,A6,A7,A8,A9,A10,A11,A12,A13,A14,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,political_views
0,6176264298,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,Moderate
1,6176263960,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Moderate
2,6176258621,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,Liberal
3,6176257082,0,1,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,Liberal
4,6176256111,0,1,0,0,1,1,1,0,0,...,0,0,1,1,0,0,0,0,0,Liberal


In [37]:
# split into X and Y
Y = merged_dfs['political_views']
X = merged_dfs.drop(['political_views'], axis=1)

print(X.shape)
print(Y.shape)

# convert to numpy arrays
X = np.array(X)

(1021, 57)
(1021,)


In [38]:
# show Y
Y.head()

0    Moderate
1    Moderate
2     Liberal
3     Liberal
4     Liberal
Name: political_views, dtype: object

In [39]:
# work with labels
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [40]:
print(encoded_Y)

[2 2 1 ... 2 2 2]


In [41]:
print(dummy_y)

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [42]:
number_inputs= len(X[0])
hidden_nodes_1 = 8
hidden_nodes_2 =8
hidden_nodes_3=8


In [43]:
# build a model
model = Sequential()
model.add(Dense(units=hidden_nodes_1, activation='relu', input_dim= number_inputs))
model.add(Dense(units=hidden_nodes_2, activation='relu'))
model.add(Dense(units=hidden_nodes_3, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 464       
                                                                 
 dense_13 (Dense)            (None, 8)                 72        
                                                                 
 dense_14 (Dense)            (None, 8)                 72        
                                                                 
 dense_15 (Dense)            (None, 5)                 45        
                                                                 
Total params: 653
Trainable params: 653
Non-trainable params: 0
_________________________________________________________________


In [44]:
# compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
              metrics=['accuracy'])

In [45]:
import keras
from keras.callbacks import EarlyStopping

# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=10, 
                                   restore_best_weights=True) # important - otherwise you just return the last weigths...



In [46]:
# now we just update our model fit call
history = model.fit(X,
                    dummy_y,
                    epochs=500, 
                    
                   
                    #batch_size=10,
                    #shuffle=True,
                    validation_split=0.2)
                    #verbose=1
                    #callbacks=[es],

Epoch 1/500
26/26 [==============================] - 1s 9ms/step - loss: 554252224.0000 - accuracy: 0.1385 - val_loss: 418016032.0000 - val_accuracy: 0.1902
Epoch 2/500
26/26 [==============================] - 0s 3ms/step - loss: 309166048.0000 - accuracy: 0.1630 - val_loss: 228648176.0000 - val_accuracy: 0.1073
Epoch 3/500
26/26 [==============================] - 0s 5ms/step - loss: 157752992.0000 - accuracy: 0.1691 - val_loss: 102466432.0000 - val_accuracy: 0.1073
Epoch 4/500
26/26 [==============================] - 0s 4ms/step - loss: 52692396.0000 - accuracy: 0.1581 - val_loss: 16075115.0000 - val_accuracy: 0.1073
Epoch 5/500
26/26 [==============================] - 0s 8ms/step - loss: 8596191.0000 - accuracy: 0.2880 - val_loss: 5058272.5000 - val_accuracy: 0.4293
Epoch 6/500
26/26 [==============================] - 0s 6ms/step - loss: 4249935.0000 - accuracy: 0.2733 - val_loss: 4025411.5000 - val_accuracy: 0.4293
Epoch 7/500
26/26 [==============================] - 0s 6ms/step - l

In [47]:

history_dict = history.history

# learning curve
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']


In [48]:
acc

[0.13848039507865906,
 0.16299019753932953,
 0.16911764442920685,
 0.15808823704719543,
 0.28799018263816833,
 0.27328431606292725,
 0.2340686321258545,
 0.2647058963775635,
 0.28921568393707275,
 0.2757352888584137,
 0.28921568393707275,
 0.2916666567325592,
 0.2904411852359772,
 0.2720588147640228,
 0.25735294818878174,
 0.30514705181121826,
 0.25735294818878174,
 0.28308823704719543,
 0.30024510622024536,
 0.2855392098426819,
 0.2720588147640228,
 0.2696078419685364,
 0.27450981736183167,
 0.3026960790157318,
 0.2708333432674408,
 0.24877451360225677,
 0.27328431606292725,
 0.31495097279548645,
 0.2769607901573181,
 0.28431373834609985,
 0.27818626165390015,
 0.24387255311012268,
 0.3075980246067047,
 0.2696078419685364,
 0.2708333432674408,
 0.2855392098426819,
 0.27450981736183167,
 0.2696078419685364,
 0.2401960790157318,
 0.2659313678741455,
 0.2647058963775635,
 0.2916666567325592,
 0.2855392098426819,
 0.26838234066963196,
 0.2757352888584137,
 0.24264705181121826,
 0.26715686

In [49]:
val_acc

[0.1902438998222351,
 0.10731707513332367,
 0.10731707513332367,
 0.10731707513332367,
 0.4292683005332947,
 0.4292683005332947,
 0.20975609123706818,
 0.20975609123706818,
 0.06341463327407837,
 0.4292683005332947,
 0.20975609123706818,
 0.4292683005332947,
 0.20975609123706818,
 0.4292683005332947,
 0.4292683005332947,
 0.4292683005332947,
 0.4292683005332947,
 0.1902438998222351,
 0.20975609123706818,
 0.4292683005332947,
 0.20975609123706818,
 0.1902438998222351,
 0.06341463327407837,
 0.1902438998222351,
 0.20975609123706818,
 0.4292683005332947,
 0.06341463327407837,
 0.20975609123706818,
 0.10731707513332367,
 0.10731707513332367,
 0.4292683005332947,
 0.1902438998222351,
 0.20975609123706818,
 0.4292683005332947,
 0.4292683005332947,
 0.20975609123706818,
 0.1902438998222351,
 0.20975609123706818,
 0.4292683005332947,
 0.4292683005332947,
 0.4292683005332947,
 0.4292683005332947,
 0.06341463327407837,
 0.4292683005332947,
 0.06341463327407837,
 0.4292683005332947,
 0.1902438998

In [50]:
# loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']


In [51]:
loss

[554252224.0,
 309166048.0,
 157752992.0,
 52692396.0,
 8596191.0,
 4249935.0,
 3580297.75,
 2509741.25,
 2409274.75,
 1240732.5,
 2318705.0,
 2230688.25,
 2346646.75,
 2798000.0,
 1570417.375,
 2091961.25,
 2505890.25,
 2588920.75,
 1844366.75,
 1370857.0,
 1798683.125,
 1852906.875,
 3257242.0,
 2681166.0,
 2347663.75,
 2902397.25,
 1944190.75,
 2971815.75,
 2208794.25,
 2743939.75,
 2582825.0,
 2741575.5,
 1834714.375,
 2189595.25,
 2244626.25,
 2394275.0,
 1662795.625,
 1739166.125,
 2755085.5,
 1732194.0,
 1659844.375,
 1934613.625,
 2672938.75,
 2870531.25,
 1942301.0,
 2848179.25,
 3007393.5,
 2140623.5,
 1698368.5,
 1969749.125,
 1934239.875,
 2930514.0,
 2186756.5,
 1396188.875,
 1681052.75,
 1719035.75,
 1548250.25,
 2856175.75,
 2241401.0,
 1786267.25,
 1902041.0,
 1525347.875,
 1763894.625,
 1697039.5,
 1576744.75,
 2215438.0,
 1992666.0,
 3118371.75,
 2174076.5,
 1208287.75,
 1713501.375,
 1339867.0,
 2064724.5,
 1338854.875,
 1657441.0,
 1042220.6875,
 1374863.625,
 19543

In [52]:
val_loss

[418016032.0,
 228648176.0,
 102466432.0,
 16075115.0,
 5058272.5,
 4025411.5,
 3212872.5,
 2554650.0,
 2965114.0,
 1679631.5,
 1246108.25,
 2848399.5,
 3437724.0,
 1888893.5,
 2830099.75,
 1735826.75,
 4024075.0,
 1661618.875,
 1975692.75,
 980017.6875,
 1244852.875,
 869115.6875,
 3946762.5,
 972146.875,
 1323445.75,
 1129408.125,
 1726557.0,
 3372362.25,
 2891357.25,
 2328081.75,
 2391714.25,
 1732796.625,
 1369856.5,
 3680454.25,
 1195717.125,
 3090683.25,
 2936978.0,
 1472477.625,
 2438010.75,
 2477316.0,
 1048570.0,
 4151045.25,
 2620845.5,
 2125951.75,
 1779555.625,
 3397684.5,
 2194389.5,
 1892542.25,
 1052719.5,
 794817.6875,
 1183553.5,
 1779839.125,
 1133941.25,
 1654689.125,
 1799089.125,
 1511536.75,
 2785606.5,
 3665147.25,
 1206745.125,
 2090485.125,
 1479644.125,
 1758777.875,
 2697546.5,
 1190632.5,
 1973249.5,
 1280587.5,
 985890.8125,
 3452803.5,
 1651734.75,
 1932141.625,
 1549688.875,
 2249499.5,
 1534017.5,
 1070965.75,
 1311307.25,
 797385.125,
 2474060.75,
 2294

In [53]:

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)
epochs



range(1, 501)

In [54]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(X) # see how the model did!



32/32 [==============================] - 0s 1ms/step


In [55]:
print(preds[0]) # i'm spreading that prediction across three nodes and they sum to 1

[1. 1. 1. 1. 1.]


In [56]:
print(np.sum(preds[0]))

5.0


In [57]:
# Almost a perfect prediction
# actual is left, predicted is top
# names can be found by inspecting Y
matrix = confusion_matrix(dummy_y.argmax(axis=1), preds.argmax(axis=1))
matrix

array([[215,   0,   0,   0,   0],
       [220,   0,   0,   0,   0],
       [420,   0,   0,   0,   0],
       [ 55,   0,   0,   0,   0],
       [111,   0,   0,   0,   0]], dtype=int64)

In [58]:
print(classification_report(dummy_y.argmax(axis=1), preds.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.21      1.00      0.35       215
           1       0.00      0.00      0.00       220
           2       0.00      0.00      0.00       420
           3       0.00      0.00      0.00        55
           4       0.00      0.00      0.00       111

    accuracy                           0.21      1021
   macro avg       0.04      0.20      0.07      1021
weighted avg       0.04      0.21      0.07      1021



c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\m099875\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


combine "very" categories? 19.3.3